## Import ##

In [ ]:
import numpy as np
import tensorflow as tf

import matplotlib.pyplot as plt
from model_profiler import model_profiler
import pathlib
from pathlib import Path
import cv2 as cv
import json
import pandas as pd
import nvidia_smi
from numba import cuda

batch_size = 20
epoch_size = 3000
test_len = 8

In [ ]:
def flatten(l):
    return [item for sublist in l for item in sublist]


def get_txt_data(path: Path) -> np.ndarray:
    with path.open("r", encoding="utf-8") as f:
        result: list = json.loads(f.readline())
        return np.array(flatten(result))


# p = pathlib.Path('../../openCV_python/data/out/resized_frames/resized')
# txt_files = list(p.glob('*.txt'))
# img_files = [item.parent.resolve() / item.stem for item in txt_files]
# txt_files[0].stem
# image = txt_files[0].parent.resolve() / txt_files[0].stem

# img_count = len(txt_files)
# img_shape = cv.imread(str(image)).shape

# txt_shape = get_txt_data(txt_files[0]).shape

# input_shape = (img_count, *img_shape)
# output_shape = (img_count, *txt_shape)
input_data: np.ndarray = np.load('data/train_input.npy')
output_data: np.ndarray = np.load('data/train_output.npy')
input_shape = input_data.shape[1:]
output_shape = output_data.shape[1:]
# data = []

# for i in range(len(img_files)):
#     input_data[i] = cv.imread(str(img_files[i]))
#     output_data[i] = get_txt_data(txt_files[i])

print('input shape')
print(input_shape)
print()
print('output shape')
print(output_shape)


def load_data():
    l = len(input_data)
    last = l - 1
    first = int(l-l/test_len)
    train_images = input_data[0:first]
    train_labels = output_data[0:first]

    test_images = input_data[first:last]
    test_labels = output_data[first:last]
    print('Train shape: {}'.format(train_images.shape))
    print('Test shape: {}'.format(test_images.shape))

    return (train_images, train_labels), (test_images, test_labels)


In [ ]:
(train_images, train_labels), (test_images, test_labels) = load_data()
input_shape = train_images.shape[1:3]


In [ ]:
activation_fun = 'gelu'
print(np.prod(input_shape))

normalization = tf.keras.layers.Normalization(axis=None)
normalization.adapt([0,255.0])

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Input(input_shape))
model.add(normalization)
model.add(tf.keras.layers.Conv1D(32, 3, activation=activation_fun))
model.add(tf.keras.layers.Conv1D(32, 3, activation=activation_fun))
model.add(tf.keras.layers.MaxPooling1D(2))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(1024 * 120, activation_fun))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(1024, activation_fun))
model.add(tf.keras.layers.Dense(8))

model.summary()


In [ ]:
# profile = model_profiler(model, batch_size)

# print(profile)


In [ ]:
model.compile(optimizer='adam',
              loss="mae",)

history = model.fit(
    train_images, train_labels,
    epochs=epoch_size,
    batch_size=batch_size,
    validation_data=(test_images, test_labels)
)


In [ ]:
history: dict = history.history
loss = history['loss']
val_loss = history['val_loss']
l = int(len(loss) / 10)
history['loss'] = loss[l:]
history['val_loss'] = val_loss[l:]


In [ ]:

print(history)
history_df = pd.DataFrame(history)
history_df.loc[:, ['loss', 'val_loss']].plot()
print("Minimum validation loss: {}".format(history_df['val_loss'].min()))

# plt.plot(history.history['accuracy'], label='accuracy')
# plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
# plt.xlabel('Epoch')
# plt.ylabel('Accuracy')
# plt.ylim([0.5, 1])
# plt.legend(loc='lower right')

# test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)


In [ ]:
# print(test_acc)

# cuda.select_device(0)
# cuda.close()


In [ ]:
model.save('out')